In [ ]:
import pandas as pd
import numpy as np
import geopandas as gpd
from shapely.geometry import Point
import os
import sys
import requests
import openmeteo_requests
import requests_cache
import pandas as pd
from retry_requests import retry
import matplotlib.pyplot as plt
import time 

from vayu_gnn.dbx.dbx_config import dbx_helper, DropboxHelper

In [ ]:
city = 'Patna'

devices = dbx_helper.read_pickle(dbx_helper.clean_input_path, 'map_device_to_latlong', f'{city}_static.pickle')

# Calculate the center (average latitude and longitude) from the existing devices
center_lat = float(sum(device['lat'] for device in devices.values()) / len(devices))
center_long = float(sum(device['long'] for device in devices.values()) / len(devices))

half_side = 1
offsets = np.linspace(-half_side, half_side, 9)  # 9 equally spaced points from -1 to 1

extra_nodes = {}
node_counter = 1

# Top side: fixed latitude (center_lat + 1), longitude varies by offset
for offset in offsets:
    device_id = f"extra_node_{node_counter}"
    lat_val = float(round(center_lat + half_side, 4))
    long_val = float(round(center_long + offset, 4))
    extra_nodes[device_id] = {'lat': lat_val, 'long': long_val}
    node_counter += 1

# Bottom side: fixed latitude (center_lat - 1), longitude varies by offset
for offset in offsets:
    device_id = f"extra_node_{node_counter}"
    lat_val = float(round(center_lat - half_side, 4))
    long_val = float(round(center_long + offset, 4))
    extra_nodes[device_id] = {'lat': lat_val, 'long': long_val}
    node_counter += 1

# Left side: fixed longitude (center_long - 1), latitude varies by offset (excluding corners)
for offset in offsets[1:-1]:
    device_id = f"extra_node_{node_counter}"
    lat_val = float(round(center_lat + offset, 4))
    long_val = float(round(center_long - half_side, 4))
    extra_nodes[device_id] = {'lat': lat_val, 'long': long_val}
    node_counter += 1

# Right side: fixed longitude (center_long + 1), latitude varies by offset (excluding corners)
for offset in offsets[1:-1]:
    device_id = f"extra_node_{node_counter}"
    lat_val = float(round(center_lat + offset, 4))
    long_val = float(round(center_long + half_side, 4))
    extra_nodes[device_id] = {'lat': lat_val, 'long': long_val}
    node_counter += 1

combined_devices = {**devices, **extra_nodes}

dbx_helper.write_pickle(combined_devices, dbx_helper.clean_input_path, f'node_locations/{city}', f'nodes.pickle')

# Also save as a gdf
records = []
for device_id, coords in combined_devices.items():
    records.append({
        'device_id': device_id,
        'lat': coords['lat'],
        'long': coords['long']
    })

df = pd.DataFrame(records)
df['geometry'] = df.apply(lambda row: Point(row['long'], row['lat']), axis=1)
gdf = gpd.GeoDataFrame(df, geometry='geometry', crs="EPSG:4326")

dbx_helper.write_shp(gdf, dbx_helper.clean_input_path, f'node_locations/{city}/gdf', f'nodes')